### 여러 pred값들 concatenate

In [ ]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer ,TfidfVectorizer
# ------------------------------------------
import torch
# ------------------------------------------
from tqdm.auto import tqdm
from sklearn.metrics import f1_score

# 라벨인코더로 트랜스폼 필수

label encoder 

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/DL_Senti_발화자/train.csv')
le = LabelEncoder()
train["target"] = le.fit_transform(train['Target'].to_numpy())

In [ ]:
le.inverse_transform(train["target"])

array(['neutral', 'neutral', 'neutral', ..., 'surprise', 'neutral', 'joy'],
      dtype=object)

## 산술평균

In [ ]:
PRED1 = np.load("/content/drive/MyDrive/DL_Senti/0fold_pred_list.np.npy")
PRED2 = np.load("/content/drive/MyDrive/DL_Senti/xlm_roberta_base_pred_list.npy")

In [ ]:
concated_preds = np.concatenate([PRED1, PRED2])

In [ ]:
concated_preds.shape

(10, 2610, 7)

In [ ]:
preds = np.mean(concated_preds, axis=0)

In [ ]:
pred_by_Mean = np.argmax(preds, axis=1)

In [ ]:
pred_by_Mean

array([6, 4, 4, ..., 3, 4, 4])

In [ ]:
submission = pd.read_csv("/content/drive/MyDrive/DL_Senti/submission.csv")
submission = submission.drop(columns='Unnamed: 0')
submission['Target'] = le.inverse_transform(pred)
submission.to_csv('name.csv', index=False)

# 하드보팅

## 넘파이로 저장된 pred값들로 할 경우, 
- 이 경우, 각 폴드 별 pred값들을 np로 저장하여 모든 모델의 5폴드 (모델 n * 5) 전부 하드보팅 진행
- argmax 실행하기 전 파일이 필요함

load pred np files

- 예시 

In [ ]:
pred_list1 = np.load("/content/drive/MyDrive/DL_Senti/xlm_roberta_base_pred_list.npy")
pred_list2 = np.load("/content/drive/MyDrive/DL_Senti/xlm_roberta_base_pred_list.npy")
pred_list3 = np.load("/content/drive/MyDrive/DL_Senti/xlm_roberta_base_pred_list.npy")
pred_list4 = np.load("/content/drive/MyDrive/DL_Senti/xlm_roberta_base_pred_list.npy")

In [ ]:
PRED1 = np.load("/content/drive/MyDrive/DL_Senti/0fold_pred_list.np.npy")
PRED2 = np.load("/content/drive/MyDrive/DL_Senti/xlm_roberta_base_pred_list.npy")

conatenate pred files

In [ ]:
concated_preds = np.concatenate([PRED1, PRED2])

In [ ]:
concated_preds = np.concatenate([PRED1, PRED2])

In [ ]:
concated_preds.shape

(10, 2610, 7)

In [ ]:
mode_lst = [ stats.mode(lst, axis = 0)[0][0] for lst in zip(*concated_preds)]

- 최빈값만 가져온 뒤 다시 array형태로변환
- array형태로 만들어주면 1차원이 추가됨


In [ ]:
mode_np = np.array(mode_lst)
mode_np.shape

- array 변환 시 불필요하게 생성된 차원 1개를 squeeze로 제거해줌

In [ ]:
pred = mode_np.squeeze()

In [ ]:
pred.shape

- argmax로 pred 추출

In [ ]:
pred =  np.argmax(mode_np, axis=1)

## 이미 argmax가 적용된 csv 파일로 진행할 떄

In [ ]:
from scipy import stats

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/Data/발화자의 감정인식 AI/csv/앙상블/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Data/발화자의 감정인식 AI/csv/앙상블


- 여기가 성우 너 csv 파일 불러온 코드

In [ ]:
pred1 = pd.read_csv("bert_base_uncased.csv")
pred2 = pd.read_csv("emoberta_base_model2.csv")
pred3 = pd.read_csv("emoberta_epoch_10.csv")
pred4 = pd.read_csv("emoberta_submit_step1500.csv")
pred5 = pd.read_csv("submit2.csv")

In [ ]:
pred1 = pd.read_csv("moaardvoting.csv")
pred2 = pd.read_csv("sungwoo5hardvoting.csv")

In [ ]:
pred1['Target']

0        neutral
1        neutral
2        neutral
3        neutral
4        neutral
          ...   
2605         joy
2606    surprise
2607       anger
2608       anger
2609     neutral
Name: Target, Length: 2610, dtype: object

- 성우 너 csv파일에 저장된 pred값은 전부 txt 타겟 클래스로 변환된 형태라서, 다시 숫자값으로 transform진행해줬어

In [ ]:
le = LabelEncoder()
pred1 = le.fit_transform(pred1['Target'].to_numpy())

In [ ]:
pred1

array([4, 4, 4, ..., 3, 0, 4])

In [ ]:
pred2 = le.fit_transform(pred2['Target'].to_numpy())
pred2

array([6, 4, 4, ..., 3, 0, 4])

In [ ]:
pred3 = le.fit_transform(pred3['Target'].to_numpy())
pred3

array([6, 4, 4, ..., 3, 2, 3])

In [ ]:
pred4 = le.fit_transform(pred4['Target'].to_numpy())
pred4

array([6, 4, 4, ..., 3, 6, 4])

In [ ]:
pred5 = le.fit_transform(pred5['Target'].to_numpy())
pred5

array([4, 4, 4, ..., 3, 4, 4])

- 그렇게 해서 한 리스트에 전부 담고, mode를 이용해서 가장 최빈값만 가져왔어

In [ ]:
lst = [pred1,pred2]

In [ ]:
lst = [pred1, pred2, pred3, pred4, pred5]

In [ ]:
mode_lst = [ stats.mode(lst, axis = 0)[0][0] for lst in zip(*lst)]

submission

In [ ]:
submission = pd.read_csv("/content/drive/MyDrive/Data/발화자의 감정인식 AI/sample_submission.csv")
#submission = submission.drop(columns='Unnamed: 0')

In [ ]:
submission['Target'] = le.inverse_transform(mode_lst)

In [ ]:
submission.to_csv('sungwoo+moa_again앙상블블.csv', index=False)